In [3]:
import pandas as pd
import numpy as np
import os
import xml.etree.ElementTree as et
from unidecode import unidecode
import matplotlib.pyplot as plt
from re import finditer
from pprint import pprint
from time import time
import pickle
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn.ensemble import GradientBoostingClassifier, GradientBoostingRegressor, RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import roc_curve

In [14]:
cd code

/Users/db/Develop/Galvanize/Knowself/code


In [16]:
from model_code import *

In [17]:
cd ..

/Users/db/Develop/Galvanize/Knowself


In [10]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [11]:
%config InlineBackend.figure_format = 'retina'

In [4]:
def load_PAN_XML_tweets(file_dir):
    """For each xml file in the directory append the file name and tweet text to a list and return it."""
    # Get the list of files in the directory
    files_e = os.listdir(file_dir)
    users = []
    for fil_e in files_e:
        # for each file in the directory that is an xml file add its name and data to the users list.
        if fil_e[-3:] == 'xml':
            tree = et.parse(file_dir+'/'+fil_e)
            root = tree.getroot()
            users.append([fil_e[:-3], []])
            for CDATA in root.findall('document'):
                users[-1][1].append(CDATA.text[:-2])
    return users

In [5]:
def camel_case_split(word):
    matches = finditer('.+?(?:(?<=[a-z])(?=[A-Z])|(?<=[A-Z])(?=[A-Z][a-z])|$)', word)
    return [m.group(0) for m in matches]

In [6]:
def featurize_tweet(tweet):
    tokens = unidecode(tweet).split()
    RT = 0
    link_ct = 0
    at_ct = 0
    hashtags = []
    tag_ct = 0
    music_share = 0
    excl_ct = 0
    ques_ct = 0
    ellipsis = 0
    I_my = 0
    We_our = 0
    you_your = 0
    loud_words = 0
    prop_names = 0
    CAP_ct = 0
    rem_tok = []
    add_tok = []
    for token in tokens:
        http_loc = token.find('http')
        if token[:2] == 'RT':
            RT += 1
            rem_tok.append(token)
        elif token[0] == '@' or (len(token) > 1 and token[1] == '@'):
            at_ct += 1
            rem_tok.append(token)
        elif http_loc != -1:
            link_ct += 1
            rem_tok.append(token)
            if http_loc != 0:
                token = token[:http_loc]
                add_tok.append(token)
        if token in {'#NowPlaying', '(Courtesy', '(Feat.', '#Shazam'}:
            music_share += 1
        if token[0] == '#' and len(token) > 1:
            if not token[1:].isdigit():
                tag_ct += 1
                hashtags.append(token)
                rem_tok.append(token)
                add_tok.extend([token[1:]])
                if camel_case_split(token[1:])[0] != token[1:]:
                    add_tok.extend(camel_case_split(token[1:]))
        elif token.lower() in {'i', "i'm", "i've", "i'd", "i'll", 'me', 'my', 'mine', 'myself'}:
            I_my += 1
        elif token.lower() in {'we', 'our', 'us', "we'd", "we've", "we'll", 'ours', 'ourselves'}:
            We_our +=1
        elif token.lower() in {'you', 'your', "you've", "you'd", "you'll", 'yours', 'yourself'}:
            you_your += 1
        if token.isupper():
            loud_words += 1
        elif token[0].isupper():
            prop_names += 1
        for c in token:
            if c == '!':
                excl_ct += 1
            if c == '?':
                ques_ct += 1
            if c == '.':
                ellipsis += 1
    for token in add_tok:
        tokens.append(token)
    for token in rem_tok:
        tokens.remove(token)
    for token in tokens:
        for c in token:
            if c.isupper():
                CAP_ct += 1
    word_ct = len(tokens)
    tw_text = ' '.join(tokens)
    features = [tweet, 1, RT, word_ct, link_ct, at_ct, hashtags, tag_ct, music_share, excl_ct, ques_ct, ellipsis, I_my, We_our, you_your, loud_words, prop_names, CAP_ct, tw_text]
    columns = ['tweet', 'tweet_ct', 'RT', 'word_ct', 'link_ct', 'at_ct', 'hashtags', 'tag_ct', 'music_share', 'excl_ct', 'ques_ct', 'ellipsis', 'I_my', 'We_our', 'you_your', 'loud_words', 'prop_names', 'CAP_ct', 'tw_text']
    return features, columns

In [33]:
X, y = main()

In [34]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Index: 304 entries, 0 to fde8eb00
Data columns (total 17 columns):
tweet_ct       152 non-null float64
RT             152 non-null float64
word_ct        152 non-null float64
link_ct        152 non-null float64
at_ct          152 non-null float64
tag_ct         152 non-null float64
music_share    152 non-null float64
excl_ct        152 non-null float64
ques_ct        152 non-null float64
ellipsis       152 non-null float64
I_my           152 non-null float64
We_our         152 non-null float64
you_your       152 non-null float64
loud_words     152 non-null float64
prop_names     152 non-null float64
CAP_ct         152 non-null float64
tw_text        152 non-null object
dtypes: float64(16), object(1)
memory usage: 42.8+ KB


In [35]:
y.info()

<class 'pandas.core.frame.DataFrame'>
Index: 304 entries, 0 to fde8eb00
Data columns (total 8 columns):
userid           152 non-null object
gender           152 non-null float64
age_group        152 non-null float64
extroverted      152 non-null float64
stable           152 non-null float64
agreeable        152 non-null float64
conscientious    152 non-null float64
open             152 non-null float64
dtypes: float64(7), object(1)
memory usage: 21.4+ KB


In [36]:
len(y)

304

In [37]:
X

,tweet_ct,RT,word_ct,link_ct,at_ct,tag_ct,music_share,excl_ct,ques_ct,ellipsis,I_my,We_our,you_your,loud_words,prop_names,CAP_ct,tw_text
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
y

,userid,gender,age_group,extroverted,stable,agreeable,conscientious,open
0,3e2cdc34,-1.0,2.0,0.3,0.5,0.1,0.2,0.2
1,b38fee80,-1.0,2.0,0.2,0.4,0.2,0.0,0.2
2,60468a75,-1.0,1.0,0.1,0.2,0.1,0.1,0.1
3,e78b60f0,1.0,1.0,0.1,0.1,-0.1,0.5,0.1
4,f1dcc4ff,1.0,1.0,0.0,-0.1,0.1,0.3,0.4
5,57af56a7,-1.0,1.0,0.3,0.3,0.2,0.1,0.2
6,ed4bf592,-1.0,1.0,0.1,0.2,0.3,0.1,0.2
7,18695d44,-1.0,1.0,0.1,-0.1,0.2,0.0,0.2
8,eb5cea6d,-1.0,1.0,0.3,0.1,0.4,0.0,0.3
9,68b27ee9,1.0,3.0,0.1,0.4,0.5,0.5,0.3
